In [1]:
import numpy as np
import math
from xml.etree.ElementTree import parse
import glob
import pprint

In [2]:
init_plan_data_path = 'C:/Users/eornr/Desktop/source_crx/200331_point_gen/01_point_gen_in_STL/05_gen_point_init_planing_data/raw_data/'

file_ext = '.xml'
xml_data_name = ['c1L_plan', 
                 'c1R_plan', 
                 'c2L_plan', 
                 'c2R_plan']

femur_imp_model_mtx = []
tibia_imp_model_mtx = []

for fi in range(len(xml_data_name)):
    tree = parse(init_plan_data_path + xml_data_name[fi] + file_ext)
    main_root = tree.getroot()

    femur_imp_model = main_root.getchildren()[2].getchildren()[1].getchildren()[0]
    tibia_imp_model = main_root.getchildren()[3].getchildren()[1].getchildren()[0]
    
    value_split_f = femur_imp_model.find("MatrixModel2Local").text.split(',')
    value_split_t = tibia_imp_model.find("MatrixModel2Local").text.split(',')
    
    femur_imp_model_mtx.append(value_split_f)
    tibia_imp_model_mtx.append(value_split_t)

In [3]:
def mtx_value_list_2_mtx_array(mtx_value_list):
    mtx_array = []
    
    count = 0
    for i in range(4):
        tmp = []
        for j in range(4):
            tmp.append(float(mtx_value_list[j + count]))
        count = count + 4
        mtx_array.append(tmp)
        
    return mtx_array

def mtx2euler(mtx) :
    dLocSqrt = (mtx[0][0] * mtx[0][0] +  mtx[1][0] * mtx[1][0])
    
    if(dLocSqrt<0.0):
        dLocSqrt = 0.0
    else:
        dLocSqrt= math.sqrt(dLocSqrt)
    
    dLocRy = math.atan2(-mtx[2][0], dLocSqrt)
    dLocCy = math.cos(dLocRy)
 
    if (dLocCy < 0.001745328365898):
        Rx = 0.0
        Ry = dLocRy
        Rz = math.atan2(-mtx[0][1], mtx[1][1])
    else:
        Rx = math.atan2(mtx[2][1]/dLocCy, mtx[2][2]/dLocCy)
        Ry = dLocRy
        Rz = math.atan2(mtx[1][0]/dLocCy, mtx[0][0]/dLocCy)
        
    X = mtx[0][3]
    Y = mtx[1][3]
    Z = mtx[2][3]
    Rx = Rx * 180 / 3.141592653589793238462643383279
    Ry = Ry * 180 / 3.141592653589793238462643383279
    Rz = Rz * 180 / 3.141592653589793238462643383279
 
    return np.array([X, Y, Z, Rx, Ry, Rz])

In [4]:
femur_imp_mtxs = []
tibia_imp_mtxs = []

for i in range(4):
    femur_imp_mtxs.append(np.array(mtx_value_list_2_mtx_array(femur_imp_model_mtx[i])))
    tibia_imp_mtxs.append(np.array(mtx_value_list_2_mtx_array(tibia_imp_model_mtx[i])))

### read sim. data!!

In [12]:
result_data_path = 'C:/Users/eornr/Desktop/source_crx/200331_point_gen/01_point_gen_in_STL/06_gen_point_data_proto/result/'
mtx_str = 'mtx'
log_str = 'log'

In [13]:
result_files = glob.glob(result_data_path + '*')

In [14]:
ans_mtx      = [[-1.00000000e+00,  9.38133875e-17, -7.87187789e-17, 6.25000000e+02],
                [ 1.22464680e-16,  7.66044443e-01, -6.42787610e-01, 1.95000000e+02],
                [ 0.00000000e+00, -6.42787610e-01, -7.66044443e-01, 2.25000000e+02],
                [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, 1.00000000e+00]]

identity_mtx = [[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, 0.00000000e+00],
                [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, 0.00000000e+00],
                [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, 0.00000000e+00],
                [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, 1.00000000e+00]]

### pose + 400 test

In [8]:
mtx_of_exp = []

for i in range(len(result_files)):
    file_path_str = result_files[i]
    if(log_str in file_path_str):
        with open(file_path_str) as file:
            for line in file:
                print(line)
    elif(mtx_str in file_path_str):
        with open(file_path_str) as file:
            for line in file:
                _line = line.split('\n')
                each_value = _line[0].split(',')
                tmp = []
                for i in range(len(each_value)):
                    tmp.append(float(each_value[i]))
                mtx_of_exp.append(tmp)
                
mtx_of_exp = np.array(mtx_of_exp)

coarse iter : 342

coarse RMSE : 0.056576

fine iter : 7

fine RMSE : 0.019404

fine RMSE all pts : 0.057446                



In [9]:
mtx2euler(np.absolute(ans_mtx)) - mtx2euler(np.absolute(np.linalg.inv(mtx_of_exp)))

array([ 0.13659469, -0.99322275, -0.66957547,  1.7221684 ,  7.09404309,
       -2.54704127])

In [10]:
indexer = 0
result = (mtx2euler(np.absolute(np.dot(ans_mtx, 
                                       femur_imp_mtxs[indexer]))) - 
          mtx2euler(np.absolute(np.dot(np.linalg.inv(mtx_of_exp), 
                                       femur_imp_mtxs[indexer]))))
result

array([-2.88980026, -0.42511994, -0.392029  ,  1.7221684 ,  7.09404309,
       -2.54704127])

###### in implant space...

In [11]:
# Te = (pose * plan)^-1 * regi. * plan
Te = np.dot(np.dot(np.linalg.inv(np.dot(ans_mtx, femur_imp_mtxs[indexer])), np.linalg.inv(mtx_of_exp)), femur_imp_mtxs[indexer])
mtx2euler(Te)

array([-2.88980026,  0.0734715 , -0.57359881, -1.78039102,  3.79688077,
       -6.51348926])

### 400 test

In [15]:
mtx_of_exp = []

for i in range(len(result_files)):
    file_path_str = result_files[i]
    if(log_str in file_path_str):
        with open(file_path_str) as file:
            for line in file:
                print(line)
    elif(mtx_str in file_path_str):
        with open(file_path_str) as file:
            for line in file:
                _line = line.split('\n')
                each_value = _line[0].split(',')
                tmp = []
                for i in range(len(each_value)):
                    tmp.append(float(each_value[i]))
                mtx_of_exp.append(tmp)
                
mtx_of_exp = np.array(mtx_of_exp)

coarse iter : 342

coarse RMSE : 0.056577

fine iter : 7

fine RMSE : 0.019405

fine RMSE all pts : 0.057446                



In [16]:
mtx2euler(np.absolute(identity_mtx)) - mtx2euler(np.absolute(np.linalg.inv(mtx_of_exp)))

array([-0.13659611, -0.33044001, -1.15136633, -1.78045159,  3.79909886,
       -6.51221767])

In [17]:
indexer = 0
result = (mtx2euler(np.absolute(np.dot(identity_mtx, 
                                       femur_imp_mtxs[indexer]))) - 
          mtx2euler(np.absolute(np.dot(np.linalg.inv(mtx_of_exp), 
                                       femur_imp_mtxs[indexer]))))
result

array([-2.88977185,  0.0736579 , -0.57359221, -1.7409175 ,  3.79909886,
       -6.51221767])

###### in implant space...

In [18]:
# Te = (pose * plan)^-1 * regi. * plan
Te = np.dot(np.dot(np.linalg.inv(np.dot(identity_mtx, femur_imp_mtxs[indexer])), np.linalg.inv(mtx_of_exp)), femur_imp_mtxs[indexer])
mtx2euler(Te)

array([-2.88980185,  0.07346001, -0.57361755, -1.78036725,  3.79685653,
       -6.51352104])

### 40 test

In [22]:
mtx_of_exp = []

for i in range(len(result_files)):
    file_path_str = result_files[i]
    if(log_str in file_path_str):
        with open(file_path_str) as file:
            for line in file:
                print(line)
    elif(mtx_str in file_path_str):
        with open(file_path_str) as file:
            for line in file:
                _line = line.split('\n')
                each_value = _line[0].split(',')
                tmp = []
                for i in range(len(each_value)):
                    tmp.append(float(each_value[i]))
                mtx_of_exp.append(tmp)
                
mtx_of_exp = np.array(mtx_of_exp)

coarse iter : 82

coarse RMSE : 0.399637

fine iter : 18

fine RMSE : 0.168117

fine RMSE all pts : 0.407744                



In [23]:
mtx2euler(np.absolute(identity_mtx)) - mtx2euler(np.absolute(np.linalg.inv(mtx_of_exp)))

array([-0.01273744, -0.03302277, -0.14926517, -0.57652166,  0.46955029,
       -0.11822609])

In [24]:
indexer = 0
result = (mtx2euler(np.absolute(np.dot(identity_mtx, 
                                       femur_imp_mtxs[indexer]))) - 
          mtx2euler(np.absolute(np.dot(np.linalg.inv(mtx_of_exp), 
                                       femur_imp_mtxs[indexer]))))
result

array([-0.15239651, -0.19226038,  0.32426435, -0.53698758,  0.46955029,
       -0.11822609])

###### in implant space...

In [25]:
# Te = (pose * plan)^-1 * regi. * plan
Te = np.dot(np.dot(np.linalg.inv(np.dot(identity_mtx, femur_imp_mtxs[indexer])), np.linalg.inv(mtx_of_exp)), femur_imp_mtxs[indexer])
mtx2euler(Te)

array([-0.15242651, -0.19214848,  0.32433064, -0.57652228,  0.46950945,
       -0.11838807])

### 자가 정합 40

coarse iter : 28  
coarse RMSE : 0.133318  
fine iter : 81  
fine RMSE : 0.006778  
fine RMSE all pts : 0.140962  

In [86]:
test_mtx_id = [[0.999998331069946,0.00172358111012727,-0.000586170644965023,-0.0312805958092213],
               [-0.00172355852555484,0.999998509883881,3.90266613976564E-05,0.0254219025373459],
               [0.000586237059906125,-3.80162964574993E-05,0.999999821186066,-0.0108767841011286],
               [0,0,0,1]]

test_mtx_id_ = np.array(test_mtx_id)

In [87]:
mtx2euler(np.absolute(identity_mtx)) - mtx2euler(np.absolute(np.linalg.inv(test_mtx_id_)))

array([-0.03133074, -0.02536836, -0.01085745, -0.00223606,  0.03358511,
       -0.09875399])

In [88]:
indexer = 0
result = (mtx2euler(np.absolute(np.dot(identity_mtx, 
                                       femur_imp_mtxs[indexer]))) - 
          mtx2euler(np.absolute(np.dot(np.linalg.inv(test_mtx_id_), 
                                       femur_imp_mtxs[indexer]))))
result

array([-0.05173189, -0.02473668,  0.01018653, -0.00223606,  0.03358511,
       -0.09875399])

###### in implant space...

In [119]:
# Te = (pose * plan)^-1 * regi. * plan
Te = np.dot(np.dot(np.linalg.inv(np.dot(identity_mtx, femur_imp_mtxs[indexer])), np.linalg.inv(test_mtx_id_)), femur_imp_mtxs[indexer])
mtx2euler(Te)

array([ 0.05173189, -0.02473316,  0.01019506,  0.00223609,  0.03361917,
        0.09874239])
